In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("SEC_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Please set it in your .env file.")

In [9]:
from sec_api import XbrlApi

xbrlApi = XbrlApi(API_KEY)

In [30]:
import pandas as pd
import zipfile

with zipfile.ZipFile("/Users/agalyaayyadurai/Documents/Dissertation/Automated-Equity-Valuation/data/sec/2025q2.zip", "r") as z:
    print(z.namelist())


['sub.txt', 'pre.txt', 'num.txt', 'tag.txt', 'readme.htm']


In [31]:
with zipfile.ZipFile("/Users/agalyaayyadurai/Documents/Dissertation/Automated-Equity-Valuation/data/sec/2025q2.zip") as z:
    with z.open("sub.txt") as f:
        sub = pd.read_csv(f, sep="\t", low_memory=False)

    with z.open("num.txt") as f:
        num = pd.read_csv(f, sep="\t", low_memory=False)

    with z.open("tag.txt") as f:
        tag = pd.read_csv(f, sep="\t", low_memory=False)

    with z.open("pre.txt") as f:
        pre = pd.read_csv(f, sep="\t", low_memory=False)
print("sub.txt:", sub.shape)
print("num.txt:", num.shape)
print("tag.txt:", tag.shape)
print("pre.txt:", pre.shape)

sub.head()

sub.txt: (7009, 36)
num.txt: (3409930, 10)
tag.txt: (89426, 9)
pre.txt: (769822, 10)


,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000950170-25-071771,1965934,OVERLAND ADVANTAGE,NaN,US,NY,NEW YORK,10152,375 PARK AVENUE,11TH FLOOR,...,20250331.0,2025.0,Q1,20250515,2025-05-14 20:36:00.0,0,1,ck0001965934-20250331_htm.xml,1,NaN
1,0000950170-25-070633,2052153,APOLLO ORIGINATION II (UL) CAPITAL TRUST,NaN,US,NY,NEW YORK,10019,"9 WEST 57TH STREET, 42ND FLOOR",NaN,...,20250331.0,2025.0,Q1,20250514,2025-05-13 18:14:00.0,0,1,ck0002052153-20250331_htm.xml,1,NaN
2,0000816956-25-000011,816956,CONMED CORP,3845.0,US,FL,LARGO,33773,11311 CONCEPT BOULEVARD,NaN,...,20250331.0,2025.0,Q1,20250501,2025-05-01 11:08:00.0,0,1,cnmd-20250331_htm.xml,1,NaN
3,0001213900-25-037427,1997201,PS INTERNATIONAL GROUP LTD.,4731.0,CN,NaN,HONG KONG,00000,"UN. 1002, 10/F, JOIN-IN HANG SING CTR.,","#2-16 KWAI FUNG CRESCENT, KWAI CHUNG, NT",...,20241231.0,2024.0,FY,20250430,2025-04-30 14:06:00.0,0,1,ea0238916-20f_psinter_htm.xml,1,NaN
4,0001679273-25-000026,1679273,"LAMB WESTON HOLDINGS, INC.",2030.0,US,ID,EAGLE,83616,599 S. RIVERSHORE LANE,NaN,...,20250228.0,2025.0,Q3,20250403,2025-04-03 13:05:00.0,0,1,lw-20250223_htm.xml,1,NaN


In [33]:
# Apple CIK (10 digits) -> in FSDS 'sub.cik' is numeric, without leading zeros
APPLE_CIK = 320193

# Filter to 10-Ks for this CIK in this quarter; pick the most recent by 'filed'
aapl_10k = (
    sub[(sub["cik"] == APPLE_CIK) & (sub["form"] == "10-Q")]
    .sort_values("filed")
    .tail(1)
)

if aapl_10k.empty:
    raise ValueError("No 10-K for AAPL in this ZIP. Try a different quarter.")

adsh = aapl_10k["adsh"].iloc[0]  # accession number (key for the filing)
fy   = aapl_10k["fy"].iloc[0]
period = aapl_10k["period"].iloc[0]  # balance sheet date (yyyymmdd)
adsh, fy, period


('0000320193-25-000057', np.float64(2025.0), np.float64(20250331.0))

In [34]:
n = num[num["adsh"] == adsh].copy()

# Keep only 'USD' (you can relax this if needed)
n = n[n["uom"] == "USD"].copy()

# For BS (instant values) you'll want qtrs == 0; for IS/CF (annual) qtrs == 4
# We’ll filter per-statement later, but it’s handy to have:
n["ddate"] = pd.to_datetime(n["ddate"].astype(str), format="%Y%m%d", errors="coerce")


In [35]:
p = pre[pre["adsh"] == adsh].copy()

# Keep only the main statements
# BS = Balance Sheet, IS = Income Statement, CF = Cash Flow,
# (EQ, CI exist too; include if you like)
p = p[p["stmt"].isin(["BS","IS","CF"])]

# We'll join PRE -> NUM on (adsh, tag, version)
cols_to_keep = ["adsh","report","line","stmt","tag","version","plabel"]
p = p[cols_to_keep].drop_duplicates()


In [36]:
# NUM has (adsh, tag, version, ddate, qtrs, uom, value, ...).
# Join on adsh, tag, version
joined = p.merge(
    n[["adsh","tag","version","ddate","qtrs","value","uom"]],
    on=["adsh","tag","version"],
    how="left"
)


In [40]:
from functools import partial

def latest_context(df, stmt_code, qtrs_needed):
    df_stmt = df[df["stmt"] == stmt_code].copy()
    df_stmt = df_stmt[df_stmt["qtrs"] == qtrs_needed].copy()
    # Keep the latest date per line (in case multiple rows exist)
    df_stmt = (
        df_stmt.sort_values(["report","line","ddate"])
               .drop_duplicates(["report","line"], keep="last")
               .sort_values(["report","line"])
    )
    return df_stmt

bs_df = latest_context(joined, "BS", 0)   # instant values
is_df = latest_context(joined, "IS", 2)   # annual duration
cf_df = latest_context(joined, "CF", 2)   # annual duration

# Tidy output columns
bs_out = bs_df[["report","line","plabel","tag","value","uom"]].reset_index(drop=True)
is_out = is_df[["report","line","plabel","tag","value","uom"]].reset_index(drop=True)
cf_out = cf_df[["report","line","plabel","tag","value","uom"]].reset_index(drop=True)

bs_out.head(), is_out.head(), cf_out.head()


(   report  line                        plabel  \
 0       4     3     Cash and cash equivalents   
 1       4     4         Marketable securities   
 2       4     5      Accounts receivable, net   
 3       4     6  Vendor non-trade receivables   
 4       4     7                   Inventories   
 
                                      tag         value  uom  
 0  CashAndCashEquivalentsAtCarryingValue  0.000000e+00  USD  
 1            MarketableSecuritiesCurrent  6.820000e+08  USD  
 2           AccountsReceivableNetCurrent  2.613600e+10  USD  
 3             NontradeReceivablesCurrent  2.366200e+10  USD  
 4                           InventoryNet  6.269000e+09  USD  ,
    report  line                               plabel  \
 0       2     7                            Net sales   
 1       2     8                        Cost of sales   
 2       2     9                         Gross margin   
 3       2    11             Research and development   
 4       2    12  Selling, general

In [41]:
def pretty_statement(df, title):
    print(f"\n=== {title} ===")
    display(
        df.rename(columns={"plabel":"Line Item","value":"Amount"})
          .assign(Amount=lambda d: pd.to_numeric(d["Amount"], errors="coerce"))
          .style.format({"Amount":"{:,.0f}"})
    )

pretty_statement(is_out, "Income Statement (Latest 10-K)")
pretty_statement(bs_out, "Balance Sheet (Latest 10-K)")
pretty_statement(cf_out, "Cash Flow Statement (Latest 10-K)")



=== Income Statement (Latest 10-K) ===


,report,line,Line Item,tag,Amount,uom
0,2,7,Net sales,RevenueFromContractWithCustomerExcludingAssessedTax,"166,674,000,000",USD
1,2,8,Cost of sales,CostOfGoodsAndServicesSold,"103,477,000,000",USD
2,2,9,Gross margin,GrossProfit,"103,142,000,000",USD
3,2,11,Research and development,ResearchAndDevelopmentExpense,"16,818,000,000",USD
4,2,12,"Selling, general and administrative",SellingGeneralAndAdministrativeExpense,"13,903,000,000",USD
5,2,13,Total operating expenses,OperatingExpenses,"30,721,000,000",USD
6,2,14,Operating income,OperatingIncomeLoss,"93,310,000,000",USD
7,2,15,"Other income/(expense), net",NonoperatingIncomeExpense,"-527,000,000",USD
8,2,16,Income before provision for income taxes,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,"71,894,000,000",USD
9,2,17,Provision for income taxes,IncomeTaxExpenseBenefit,"10,784,000,000",USD



=== Balance Sheet (Latest 10-K) ===


,report,line,Line Item,tag,Amount,uom
0,4,3,Cash and cash equivalents,CashAndCashEquivalentsAtCarryingValue,0,USD
1,4,4,Marketable securities,MarketableSecuritiesCurrent,"682,000,000",USD
2,4,5,"Accounts receivable, net",AccountsReceivableNetCurrent,"26,136,000,000",USD
3,4,6,Vendor non-trade receivables,NontradeReceivablesCurrent,"23,662,000,000",USD
4,4,7,Inventories,InventoryNet,"6,269,000,000",USD
5,4,8,Other current assets,OtherAssetsCurrent,"14,109,000,000",USD
6,4,9,Total current assets,AssetsCurrent,"118,674,000,000",USD
7,4,11,Marketable securities,MarketableSecuritiesNoncurrent,"9,876,000,000",USD
8,4,12,"Property, plant and equipment, net",PropertyPlantAndEquipmentNet,"46,876,000,000",USD
9,4,13,Other non-current assets,OtherAssetsNoncurrent,"81,259,000,000",USD



=== Cash Flow Statement (Latest 10-K) ===


,report,line,Line Item,tag,Amount,uom
0,7,3,Net income,NetIncomeLoss,"61,110,000,000",USD
1,7,5,Depreciation and amortization,DepreciationDepletionAndAmortization,"5,741,000,000",USD
2,7,6,Share-based compensation expense,ShareBasedCompensation,"6,512,000,000",USD
3,7,7,Other,OtherNoncashIncomeExpense,"2,217,000,000",USD
4,7,9,"Accounts receivable, net",IncreaseDecreaseInAccountsReceivable,"-7,266,000,000",USD
5,7,10,Vendor non-trade receivables,IncreaseDecreaseInOtherReceivables,"-9,171,000,000",USD
6,7,11,Inventories,IncreaseDecreaseInInventories,"-858,000,000",USD
7,7,12,Other current and non-current assets,IncreaseDecreaseInOtherOperatingAssets,"4,371,000,000",USD
8,7,13,Accounts payable,IncreaseDecreaseInAccountsPayable,"-14,604,000,000",USD
9,7,14,Other current and non-current liabilities,IncreaseDecreaseInOtherOperatingLiabilities,"-15,579,000,000",USD


In [42]:
tag_map = {
    "revenue": ["Revenues","SalesRevenueNet","RevenueFromContractWithCustomerExcludingAssessedTax"],
    "operating_income": ["OperatingIncomeLoss"],
    "net_income": ["NetIncomeLoss"],
    "assets": ["Assets"],
    "liabilities": ["Liabilities"],
    "equity": ["StockholdersEquity"],
    "cash": ["CashAndCashEquivalentsAtCarryingValue"],
    "ocf": ["NetCashProvidedByUsedInOperatingActivities"],
    "capex": ["PaymentsToAcquirePropertyPlantAndEquipment"],
}

def pick_first_available(df, candidates):
    m = df[df["tag"].isin(candidates)]
    if m.empty: return None
    # take the last (already sorted by date)
    return pd.to_numeric(m["value"], errors="coerce").dropna().iloc[-1] if not m.empty else None

compact = {
    "adsh": adsh,
    "fy": fy,
    "bs_date": period
}

for k, tags in tag_map.items():
    # choose the appropriate source table
    if k in ["assets","liabilities","equity","cash"]:
        source = bs_df
    elif k in ["ocf","capex"]:
        source = cf_df
    else:
        source = is_df
    compact[k] = pick_first_available(source, tags)

import numpy as np
if compact.get("ocf") is not None and compact.get("capex") is not None:
    compact["fcf"] = compact["ocf"] - compact["capex"]
else:
    compact["fcf"] = np.nan

pd.DataFrame([compact])


,adsh,fy,bs_date,revenue,operating_income,net_income,assets,liabilities,equity,cash,ocf,capex,fcf
0,0000320193-25-000057,2025.0,20250331.0,1.666740e+11,9.331000e+10,6.111000e+10,3.312330e+11,2.644370e+11,6.679600e+10,0.0,5.388700e+10,6.011000e+09,4.787600e+10


In [27]:
apple_cik = "0000320193"

apple_sub = sub[sub["cik"] == int(apple_cik)]
apple_sub[["adsh", "name", "form", "fy", "filed"]]


,adsh,name,form,fy,filed
1222,0000320193-25-000057,APPLE INC,10-Q,2025.0,20250502


In [28]:
apple_num = num[num["adsh"].isin(apple_sub["adsh"])]
apple_num.head()


,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote
120833,0000320193-25-000057,AccountsReceivableNetCurrent,us-gaap/2024,20240930,0,USD,NaN,NaN,3.341000e+10,NaN
120834,0000320193-25-000057,AccumulatedOtherComprehensiveIncomeLossNetOfTax,us-gaap/2024,20250331,0,USD,NaN,NaN,-6.363000e+09,NaN
120835,0000320193-25-000057,AdjustmentsRelatedToTaxWithholdingForShareBase...,us-gaap/2024,20240331,2,USD,EquityComponents=RetainedEarnings;,NaN,1.089000e+09,NaN
120836,0000320193-25-000057,Assets,us-gaap/2024,20240930,0,USD,NaN,NaN,3.649800e+11,NaN
120837,0000320193-25-000057,CashAndCashEquivalentsAtCarryingValue,us-gaap/2024,20250331,0,USD,FairValueByFairValueHierarchyLevel=FairValueIn...,NaN,0.000000e+00,NaN


In [29]:
num["tag"].value_counts().head(10)


tag
InvestmentOwnedAtFairValue                                                131720
InvestmentOwnedAtCost                                                     113451
StockholdersEquity                                                        109913
InvestmentOwnedBalancePrincipalAmount                                      81977
RevenueFromContractWithCustomerExcludingAssessedTax                        73337
InvestmentBasisSpreadVariableRate                                          69631
StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest     69078
InvestmentOwnedPercentOfNetAssets                                          65880
InvestmentInterestRate                                                     61133
NetIncomeLoss                                                              50950
Name: count, dtype: int64

In [21]:
import os, sys, json ,requests
import pandas as pd
from datetime import datetime

HEADERS = {"User-Agent": "Agalya ayyadurai.agalya@gmail.com"}
BASE = "https://sec.gov"

def get_company_tickers():
    url = f"{BASE}/files/company_tickers.json"
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    data=r.json()
    df = pd.DataFrame.from_dict(data, orient="index")
    df["cik"] = df["cik_str"].astype(str).str.zfill(10)
    return df[["ticker", "cik", "title"]]

tickers_df = get_company_tickers()
tickers_df.head(5)
    
    

,ticker,cik,title
0,NVDA,0001045810,NVIDIA CORP
1,MSFT,0000789019,MICROSOFT CORP
2,AAPL,0000320193,Apple Inc.
3,GOOGL,0001652044,Alphabet Inc.
4,AMZN,0001018724,AMAZON COM INC


### Archived

In [6]:
import requests

# Getting the metadata for Russell-3000 constituents
# https://sec-api.io/resources/download-10-k-filings-of-russell-3000-companies-from-sec-edgar

url = 'https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?fileType=csv&fileName=IWV_holdings&dataType=fund'
response = requests.get(url)

with open('russell-3000.csv', 'wb') as f:
    f.write(response.content)

In [7]:
# cleaning CSV file
import csv

with open('russell-3000.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

empty_row_indicies = [i for i in range(len(rows)) if (len(rows[i]) == 0 or '\xa0' in rows[i])]

print('Empty rows:', empty_row_indicies)

start = empty_row_indicies[0] + 1
end = empty_row_indicies[1]
cleaned_rows = rows[start:end]

with open('russell-3000-clean.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(cleaned_rows)

import pandas as pd

# load Russell 3000 holdings CSV into a dataframe
holdings = pd.read_csv('./russell-3000-clean.csv')

print('Russell 3000 Constituents')


Empty rows: [8, 2599]
Russell 3000 Constituents


,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Quantity,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,NVDA,NVIDIA CORP,Information Technology,Equity,"1,157,651,503.92",6.70,"1,157,651,503.92","6,256,223.00",185.04,United States,NASDAQ,USD,1.0,USD,-
1,MSFT,MICROSOFT CORP,Information Technology,Equity,"1,038,792,969.90",6.01,"1,038,792,969.90","1,982,505.00",523.98,United States,NASDAQ,USD,1.0,USD,-
2,AAPL,APPLE INC,Information Technology,Equity,"999,523,591.36",5.78,"999,523,591.36","3,897,082.00",256.48,United States,NASDAQ,USD,1.0,USD,-
3,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"568,693,598.72",3.29,"568,693,598.72","2,564,224.00",221.78,United States,NASDAQ,USD,1.0,USD,-
4,META,META PLATFORMS INC CLASS A,Communication,Equity,"416,184,150.44",2.41,"416,184,150.44","583,643.00",713.08,United States,NASDAQ,USD,1.0,USD,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,-,OMNIAB INC $15.00 VESTING Prvt,Health Care,Equity,0.02,0.00,0.02,"1,502.00",0.00,United States,NO MARKET (E.G. UNLISTED),USD,1.0,USD,-
2585,-,OMNIAB INC $12.50 VESTING Prvt,Health Care,Equity,0.02,0.00,0.02,"1,502.00",0.00,United States,NO MARKET (E.G. UNLISTED),USD,1.0,USD,-
2586,GME WS,GAMESTOP EQUITY WARRANT EXPIRY TH,Consumer Discretionary,Equity,0.11,0.00,0.11,"10,864.00",0.00,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
2587,RTYZ5,RUSSELL 2000 EMINI CME DEC 25,Cash and/or Derivatives,Futures,0.00,0.00,"2,844,525.00",23.00,"2,473.50",-,Chicago Mercantile Exchange,USD,1.0,USD,-


In [16]:
holdings_it_sector = holdings[holdings['Sector']=='Information Technology']

# create batches of tickers: [[A,B,C], [D,E,F], ...]
# a single batch has a maximum of max_length_of_batch tickers
def create_batches(tickers = [], max_length_of_batch = 10):
  batches = [[]]

  for ticker in tickers:
    if len(batches[len(batches)-1]) == max_length_of_batch:
      batches.append([])

    batches[len(batches)-1].append(ticker)

  return batches


batches = create_batches(list(holdings_it_sector['Ticker']))
batches = batches[0:1]
print(batches)

[['NVDA', 'MSFT', 'AAPL', 'AVGO', 'ORCL', 'PLTR', 'AMD', 'CSCO', 'IBM', 'CRM']]


In [17]:
from sec_api import QueryApi, RenderApi
from pathlib import Path
import multiprocessing

queryApi = QueryApi(api_key=API_KEY)

def download_10K_metadata(tickers = [], start_year = 2022, end_year = 2024):
  # if Path('metadata.csv').is_file():
  #   print('✅ Reading metadata from metadata.csv')
  #   result = pd.read_csv('metadata.csv')
  #   return result

  print('✅ Starting downloading metadata for years {} to {}'.format(start_year, end_year))

  # create ticker batches, with 25 tickers per batch
  batches = create_batches(tickers)
  frames = []

  for year in range(start_year, end_year + 1):
    for batch in batches:
      tickers_joined = ', '.join(batch)
      ticker_query = f'ticker:({tickers_joined})'
      filed_at_query = f'filedAt:[{year}-01-01 TO {year}-12-31]'
      form_type_query = 'formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'

      query_string = ticker_query + ' AND '  + filed_at_query + ' AND ' + form_type_query

      query = {
        "query": query_string,
        "from": "0",
        "size": "50",
        "sort": [{ "filedAt": { "order": "desc" } }]
      }

      response = queryApi.get_filings(query)

      filings = response['filings']

      metadata = list(map(lambda f: {'ticker': f['ticker'],
                                     'cik': f['cik'],
                                     'formType': f['formType'],
                                     'filedAt': f['filedAt'],
                                     'filingUrl': f['linkToFilingDetails']}, filings))

      df = pd.DataFrame.from_records(metadata)

      frames.append(df)

    print('✅ Downloaded metadata for year', year)


  result = pd.concat(frames)
  result.to_csv('metadata.csv', index=False)

  number_metadata_downloaded = len(result)
  print('✅ Download completed. Metadata downloaded for {} filings.'.format(number_metadata_downloaded))

  return result


tickers = ['NVDA', 'MSFT', 'AAPL', 'AVGO', 'ORCL', 'PLTR', 'AMD', 'CSCO', 'IBM', 'CRM']

metadata = download_10K_metadata(tickers=tickers, start_year=2022, end_year=2024)
print('SEC form types:', list(metadata.formType.unique()))
print('Number of 10-K URLs:', len(metadata))
print('Metadata of the first 10 filings:')
metadata.head(10)

✅ Starting downloading metadata for years 2022 to 2024


Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

In [18]:

import pandas as pd 

# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 


income_statement_google = get_income_statement(xbrl_json)


print("Income statement from Google's 2022 10-K filing as dataframe")
print('------------------------------------------------------------')
income_statement_google

Income statement from Google's 2022 10-K filing as dataframe
------------------------------------------------------------


,2020-01-01-2020-12-31,2021-01-01-2021-12-31,2022-01-01-2022-12-31
RevenueFromContractWithCustomerExcludingAssessedTax,182527000000,257637000000,282836000000
CostOfRevenue,84732000000,110939000000,126203000000
ResearchAndDevelopmentExpense,27573000000,31562000000,39500000000
SellingAndMarketingExpense,17946000000,22912000000,26567000000
GeneralAndAdministrativeExpense,11052000000,13510000000,15724000000
CostsAndExpenses,141303000000,178923000000,207994000000
OperatingIncomeLoss,41224000000,78714000000,74842000000
NonoperatingIncomeExpense,6858000000,12020000000,-3514000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,48082000000,90734000000,71328000000
IncomeTaxExpenseBenefit,7813000000,14701000000,11356000000
